# Agent Development Notebook

This notebook provides an interactive environment for developing LangGraph agents with CopilotKit.

## Setup
1. Run the **Start Backend Server** cell to launch the Python AG-UI server
2. Run the **Define Agent Graph** cell to configure your agent
3. Run the **Customize Frontend** cell to configure the UI
4. Run the **Start Frontend** cell to launch the Next.js UI
5. Open http://localhost:3000 to interact with your agent

## Hot Reload
- Re-run **Define Agent Graph** to update agent behavior (no restart needed)
- Re-run **Customize Frontend** to update the UI (Next.js hot-reloads automatically)

## Start Backend Server
Run this cell once to start the FastAPI server on port 8000.

In [ ]:
import threading
import uvicorn
from ag_ui_langgraph import add_langgraph_fastapi_endpoint
from copilotkit import LangGraphAGUIAgent
from fastapi import FastAPI

app = FastAPI()

# Create agent with placeholder graph (will be set in next cell)
agent = LangGraphAGUIAgent(
    name="sample_agent",
    description="An example agent.",
    graph=None,
)

add_langgraph_fastapi_endpoint(app=app, agent=agent, path="/")

# Start server in background thread
threading.Thread(
    target=lambda: uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning"),
    daemon=True,
).start()

print("✓ Backend server running at http://localhost:8000")

## Define Agent Graph
Edit this cell to customize your agent. Re-run to hot-reload changes.

In [ ]:
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI


async def chat(state: MessagesState):
    """Main chat node - customize this function to change agent behavior."""
    model = ChatOpenAI(model="gpt-4o-mini")
    system_message = SystemMessage(content="You are a helpful assistant.")
    response = await model.ainvoke([system_message, *state["messages"]])
    return {"messages": response}


# Build the graph
memory = MemorySaver()
graph = StateGraph(MessagesState)
graph.add_node("chat", chat)
graph.add_edge(START, "chat")
graph.add_edge("chat", END)

# Update the agent's graph (hot reload)
agent.graph = graph.compile(checkpointer=memory)
print("✓ Agent graph updated!")

## Customize Frontend
Edit this cell to customize the chat UI. Re-run to update (Next.js will hot-reload).

In [ ]:
%%writefile frontend/src/app/page.tsx
"use client";

import { CopilotKitProvider, CopilotChat } from "@copilotkit/react-core/v2";
import "@copilotkit/react-core/v2/styles.css";

export const dynamic = "force-dynamic";

export default function Home() {
  return (
    <CopilotKitProvider runtimeUrl="/api/copilotkit">
      <div className="h-screen w-screen">
        <CopilotChat />
      </div>
    </CopilotKitProvider>
  );
}

## Start Frontend
Run this cell to start the Next.js frontend. This runs in the background.

In [ ]:
%%bash --bg
cd frontend && npm run dev